In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('LR_crew_cruise').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
input_file_path="file:///C:/Users/ckp43_000/Documents/cruise_info.csv"

In [5]:
df=spark.read.csv(input_file_path,inferSchema=True,header=True)

In [6]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
df.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)]

In [9]:
for ship in df.head(5):
    print(ship)
    print('n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)
n
Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)
n
Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)
n
Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)
n
Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)
n


In [11]:
df.groupBy('cruise_line').count().show()

+-----------------+-----+
|      cruise_line|count|
+-----------------+-----+
|          Azamara|    2|
|         Carnival|   22|
|        Celebrity|   10|
|            Costa|   11|
|          Crystal|    2|
|           Cunard|    3|
|           Disney|    2|
| Holland_American|   14|
|              MSC|    8|
|        Norwegian|   13|
|          Oceania|    3|
|           Orient|    1|
|              P&O|    6|
|         Princess|   17|
|Regent_Seven_Seas|    5|
|  Royal_Caribbean|   23|
|         Seabourn|    3|
|        Silversea|    4|
|             Star|    6|
|         Windstar|    3|
+-----------------+-----+



In [12]:
from pyspark.ml.feature import StringIndexer

In [15]:
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')

In [16]:
indexed=indexer.fit(df).transform(df)

In [17]:
indexed.head(5)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)]

In [18]:
from pyspark.ml.linalg import Vector

In [19]:
from pyspark.ml.feature import VectorAssembler

In [21]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [28]:
assembler=VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'],outputCol='features')

In [29]:
output=assembler.transform(indexed)

In [30]:
output.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0| 

In [33]:
final_data=output.select('features','crew')

In [34]:
final_data.show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [36]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [40]:
train_data.describe().show(5)

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               111|
|   mean| 7.606396396396407|
| stddev|3.7199412978732944|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [41]:
test_data.describe().show(5)

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                47|
|   mean| 8.237659574468085|
| stddev|2.9182701939795392|
|    min|              0.88|
|    max|              13.6|
+-------+------------------+



In [42]:
from pyspark.ml.regression import LinearRegression

In [49]:
ship_lr=LinearRegression(labelCol='crew')

In [51]:
trained_ship_model=ship_lr.fit(train_data)

In [52]:
ship_results=trained_ship_model.evaluate(test_data)

In [53]:
ship_results.rootMeanSquaredError

0.7324579167773716

In [54]:
ship_results.r2

0.9356343169697149

In [55]:
ship_results.meanSquaredError

0.536494599849847

In [56]:
ship_results.meanAbsoluteError

0.5939511232344005

In [57]:
from pyspark.sql.functions import corr

In [60]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [62]:
df.select(corr('passengers','crew')).show()

+----------------------+
|corr(passengers, crew)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [63]:
df.select(corr('cabins','crew')).show()

+------------------+
|corr(cabins, crew)|
+------------------+
|0.9508226063578497|
+------------------+

